# Exercise
## 1 SetUp

In [1]:
width = 28
height = 28
channel = 1
n_inputs = width * height

In [2]:
conv1_fmaps = 32
conv1_filter_size = 3
conv1_stride = 1
conv1_padding = 'SAME'

In [3]:
pool1_fmap = conv1_fmaps

In [4]:
conv2_fmaps = 64
conv2_filter_size = 5
conv2_stride = 1
conv2_padding = 'SAME'

In [5]:
pool2_fmap = conv2_fmaps

In [6]:
conv3_fmaps = 128
conv3_filter_size = 7
conv3_stride = 1
conv3_padding = 'SAME'

In [7]:
pool3_fmap = conv3_fmaps

In [8]:
conv4_fmaps = 64
conv4_filter_size = 5
conv4_stride = 1
conv4_padding = 'SAME'

In [9]:
pool4_fmap = conv4_fmaps

In [10]:
n_fc1 = 32
n_output = 10

## 2 Construct Graph

In [11]:
import os
import numpy as np
import tensorflow as tf

In [12]:
with tf.name_scope('input'):
    X = tf.placeholder(tf.float32, shape=(None, n_inputs), name='X')
    X_reshaped = tf.reshape(X, shape=(-1, height, width, channel))
    y = tf.placeholder(tf.int32, shape=(None), name='y')

In [13]:
with tf.name_scope('layer1'):
    conv1 = tf.layers.conv2d(X_reshaped, filters=conv1_fmaps, kernel_size=conv1_filter_size,
                    strides=conv1_stride, padding=conv1_padding,activation=tf.nn.relu, name='conv_1')
    pool1 = tf.nn.max_pool(conv1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='VALID', name='pool_1')

In [14]:
with tf.name_scope('layer2'):
    conv2 = tf.layers.conv2d(pool1, filters=conv2_fmaps, kernel_size=conv2_filter_size,
                             strides=conv2_stride, padding=conv2_padding, activation=tf.nn.relu, name='conv_2')
    pool2 = tf.nn.max_pool(conv2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='VALID', name='pool_2')

In [15]:
with tf.name_scope('layer3'):
    conv3 = tf.layers.conv2d(pool2, filters=conv3_fmaps, kernel_size=conv3_filter_size,
                             strides=conv3_stride, padding=conv3_padding, activation=tf.nn.relu, name='conv_3')
    pool3 = tf.nn.max_pool(conv3, ksize=[1,2,2,1], strides=[1,2,2,1], padding='VALID', name='pool_3')

In [16]:
with tf.name_scope('layer4'):
    conv4 = tf.layers.conv2d(pool3, filters=conv4_fmaps, kernel_size=conv4_filter_size,
                             strides=conv4_stride, padding=conv4_padding, activation=tf.nn.relu, name='conv_4')
    pool4 = tf.nn.max_pool(conv4, ksize=[1,2,2,1], strides=[1,2,2,1], padding='VALID', name='pool_4')

In [17]:
with tf.name_scope('output'):
    pool4_flat = tf.reshape(pool4, shape=[-1, conv4_fmaps * 10 * 10], name='poolflat_')
    fc1_dense = tf.layers.dense(pool4_flat, n_fc1, activation=tf.nn.relu, name='fc1dense_')
    logits = tf.layers.dense(fc1_dense, n_output, name='o_')
    Y_proba = tf.nn.log_softmax(logits, name='Y_proba_')

In [18]:
with tf.name_scope('train'):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy)
    optimzer = tf.train.AdamOptimizer()
    training_op = optimzer.minimize(loss)

In [19]:
with tf.name_scope('eval'):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [20]:
with tf.name_scope('init_and_save'):
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()

## 3 Load-Data

In [21]:
from sklearn.datasets import fetch_mldata
mnist = fetch_mldata('MNIST original')

In [35]:
data = mnist.data.astype(np.float32)
target = mnist.target.astype(np.int32)

In [36]:
X_train = data[:60000]
X_test = data[60000:]
y_train = target[:60000]
y_test = target[60000:]

In [37]:
def next_batch(n_epoch,n_iteration, batch_size):
    np.random.seed(n_epoch * n_iteration + 42)
    idx = np.random.permutation(60000)[:batch_size]
    return X_train[idx], y_train[idx]

In [38]:
n_epochs = 10
batch_size = 100
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for batch in range(60000//batch_size):
            X_batch, y_batch = next_batch(epoch, batch, batch_size)
            sess.run(training_op, feed_dict={X:X_batch, y:y_batch})
        acc_train = accuracy.eval(feed_dict={X:X_batch, y:y_batch})
        acc_test = accuracy.eval(feed_dict={X:X_test, y:y_test})
        print(epoch, "training accuracy", acc_train, "Test accuracy", acc_test)
        save_path = saver.save(sess, './my_mnist_model')

InvalidArgumentError: logits and labels must have the same first dimension, got logits shape [1,10] and labels shape [100]
	 [[Node: train/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits = SparseSoftmaxCrossEntropyWithLogits[T=DT_FLOAT, Tlabels=DT_INT32, _device="/job:localhost/replica:0/task:0/cpu:0"](output/o_/BiasAdd, _recv_input/y_0)]]

Caused by op 'train/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits', defined at:
  File "/Users/gaufung/anaconda/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/gaufung/anaconda/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/gaufung/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Users/gaufung/anaconda/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/gaufung/anaconda